# Домашняя работа по лекции №6 в которой я абсолютно ничего не понял! :( 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

In [3]:
df_gender = pd.read_csv('data/customers_gender_train.csv')
df_gender.head()

,customer_id,gender
0,75562265,0
1,10928546,1
2,69348468,1
3,84816985,1
4,61009479,0


In [4]:
df_transactions_ = pd.read_csv('data/transactions.csv.gz')
df_transactions_.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [5]:
df_tr = pd.read_csv('data/tr_types.csv', sep=';')

In [6]:
df_tr

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...
5,4010,Плата за получение наличных в АТМ Сбербанка
6,4011,Плата за получение наличных в АТМ Сбербанка (в...
7,4020,Плата за получение наличных через POS в Сбербанке
8,4021,Плата за получение наличных через POS в Сберба...
9,4030,Плата за взнос наличных через АТМ (в своем тер...


In [7]:
df_mcc = pd.read_csv('data/tr_mcc_codes.csv', sep=';')

In [8]:
df_mcc

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело
5,3000,"Авиалинии, авиакомпании"
6,3351,Агентства по аренде автомобилей
7,3501,"Жилье — отели, мотели, курорты"
8,4111,Транспортировка — пригородные и локальные сезо...
9,4112,Пассажирские железные перевозки


In [9]:
from pandas import Timestamp, DateOffset

In [10]:
def preproc_transactions(df_transactions):
    sec_per_day = 86400
    sec_per_hour = 3600
    
    start_date = 1420070400 - 154 * sec_per_day - 3 * sec_per_hour
    
    df_transactions.loc[:, 'day'] = df_transactions["tr_datetime"]\
                                               .str.split(' ')\
                                               .str.get(0)\
                                               .astype(int)
    df_transactions.loc[:, 'time_raw'] = df_transactions["tr_datetime"]\
                                                    .str.split(' ')\
                                                    .str.get(1)

    # set temp dt
    df_transactions.loc[:, 'dt_temp'] = pd.to_datetime(df_transactions.loc[:, 'time_raw'], 
                                                    format='%H:%M:%S')\
                                        + DateOffset(years=115)
    
    df_transactions = df_transactions.assign(dt = lambda x: x.dt_temp.astype(np.int64) // 10**9
                                             + (x.day - 153) * sec_per_day)\
                                     .assign(weekday = lambda x: ((x.day + 4) % 7 + 1))
        
    df_transactions.loc[:, 'datetime'] = pd.to_datetime(df_transactions.dt, unit='s')
    df_transactions.loc[:, 'date'] = df_transactions.loc[:, 'datetime'].dt.strftime('%Y-%m-%d')
    df_transactions.loc[:, 'hour'] = df_transactions.loc[:, 'datetime'].dt.strftime('%H')
    
    df_transactions = df_transactions.drop(['dt_temp', 'time_raw', 'tr_datetime'], axis=1)
    
    df_transactions.loc[:, 'amount'] = np.round(df_transactions.loc[:, 'amount']/(np.pi**np.exp(1)))
            
    return df_transactions

In [11]:
df_transactions = df_transactions_.pipe(preproc_transactions)

In [13]:
df_transactions.head()

,customer_id,mcc_code,tr_type,amount,term_id,day,dt,weekday,datetime,date,hour
0,39026145,4814,1030,-100.0,NaN,0,1406888606,5,2014-08-01 10:23:26,2014-08-01,10
1,39026145,6011,7010,2500.0,NaN,1,1406974769,6,2014-08-02 10:19:29,2014-08-02,10
2,39026145,4829,2330,-2500.0,NaN,1,1406974856,6,2014-08-02 10:20:56,2014-08-02,10
3,39026145,5499,1010,-62.0,NaN,1,1406975994,6,2014-08-02 10:39:54,2014-08-02,10
4,39026145,5499,1010,-41.0,NaN,2,1407080022,7,2014-08-03 15:33:42,2014-08-03,15


In [14]:
def gen_features(gen_features, df_transactions, df_mcc, df_tr):
#     regex = 'магазины|Универмаги|Аксессуары|одежда|обувь|для дома|общего назначения'
#     df_mcc_tmp = df_mcc.copy()
#     df_mcc_tmp.loc[:,'shopping'] = df_mcc['mcc_description'].str.contains(regex)

#     df_featchers = df_transactions.join(df_mcc_tmp, on='mcc_code', how='left')
#     return df_featchers
#     df_mcc_counts = df_transactions.pivot_table(
#             index=['customer_id'], columns=['mcc_code'], values='amount', 
#             aggfunc=np.size, fill_value=0)
#     df_mcc_counts = df_mcc_counts.rename(lambda x: 'mcc_{}_count'.format(x), axis=1)
    
#     df_mcc_counts = df_mcc_counts.rename(lambda x: 'mcc_{}_count'.format(x), axis=1) 
#     df_mcc_counts.loc[:,'shopping'] = df_mcc['mcc_description'].str.contains(regex)
#     return df_mcc_tmp
#     Взнос наличных
#     Выдача наличных
#     Наличные
#     Оплата услуг
#     Оплата услуги
#     Перевод на карту
#     Перевод с карты на карту
#     Перевод средств с карты
#     Перевод/пополнение со счета на карту
#     Платеж с карты
#     Списание с карты

#     regex_tr = 'Оплата|Наличные|наличных|Перевод|пополнение|Платеж|Списание с карты'
#     df_tr_tmp =  df_tr[df_tr['tr_description'].str.contains(regex_tr) == True]
    df_transactions_mcc = df_transactions.join(
        df_mcc.set_index('mcc_code'), 
        on='mcc_code'
    )
    
    df_transactions_mcc_pivot = df_transactions_mcc.pivot_table(
        index=['customer_id'], 
        values='mcc_description', 
        aggfunc=lambda x: ' '.join(x)
    )
    
#     return df_transactions_mcc_pivot
    
#     df_tr_counts = df_transactions\
#         .pivot_table(
#             index=['customer_id'], columns=['tr_type'], values='amount', 
#             aggfunc=np.size, fill_value=0)
        
#     df_tr_counts = df_tr_counts.rename(lambda x: 'tr_{}_count'.format(x), axis=1)
    
    
    df_weekday_rations = \
        df_transactions.pivot_table(index=['customer_id'], columns='weekday', values='amount', 
                             aggfunc=np.size, fill_value=0)
    
    total = df_weekday_rations.sum(axis=1)
    df_weekday_rations.loc[:, 'saturday'] = (df_weekday_rations.iloc[:, 5:6].sum(axis=1).T/total).T
    df_weekday_rations.loc[:, 'sunday'] = (df_weekday_rations.iloc[:, 6:7 ].sum(axis=1).T/total).T
    df_weekday_rations.loc[:, 'friday'] = (df_weekday_rations.iloc[:, 4:5].sum(axis=1).T/total).T
    df_weekday_rations.loc[:, 'weekdays'] = (df_weekday_rations.iloc[:, :4].sum(axis=1).T/total).T
    
    df_features = df_gender.join(df_weekday_rations.loc[:, ['saturday', 'sunday', 'friday', 'weekdays']], on='customer_id', how='left')\
                         .join(df_transactions_mcc_pivot, on='customer_id', how='left')\
#                                  .join(df_tr_counts, on='customer_id', how='left')\
#                                  .join(df_mcc_counts, on='customer_id', how='left')\
    return df_features

In [15]:
df_features = df_gender.pipe(
    gen_features, 
    df_transactions, 
    df_mcc, 
    df_tr
)

In [16]:
df_features.head()

,customer_id,gender,saturday,sunday,friday,weekdays,mcc_description
0,75562265,0,0.110759,0.123418,0.189873,0.575949,Финансовые институты — снятие наличности автом...
1,10928546,1,0.102351,0.107884,0.156293,0.633472,Дилеры по продаже печатной продукции Места общ...
2,69348468,1,0.130137,0.127854,0.173516,0.568493,"Горючее топливо — уголь, нефть, разжиженный бе..."
3,84816985,1,0.144330,0.103093,0.061856,0.690722,Финансовые институты — снятие наличности автом...
4,61009479,0,0.123142,0.106157,0.159236,0.611465,Финансовые институты — снятие наличности автом...


In [17]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, rand

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer


In [54]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, col_idx):
        self.col_idx = col_idx
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[:, self.col_idx]

In [55]:
label = 'gender'
RND_SEED = 123
idx_features = df_features.columns != label

In [56]:
X = df_features.loc[:, idx_features].values
y = df_features.loc[:, ~idx_features].values.flatten()

In [82]:
def run_trials_template(X, y, params, evals=100):

    def hyperopt_cv(X, y, params):
        
        X_ = X.copy()
        
        # Отделяем параметры лог регрессии в отдельный словарь
        lm_params = {}
        for k, v in params.items():
            if k.startswith('glob'):
                continue                
            elif k.startswith('lm'):
                lm_params[k.split('_', 1)[1]] = v
        
        # Задаем шкалирование
        if params['scaler_type'] == 'standart':
            scaler = StandardScaler(with_mean=params['scaler_centering'])
        else:
            assert params['scaler_type'] == 'robust'
            scaler = RobustScaler(with_centering=params['scaler_centering'])
        
        # Создаем лог рег с нужными параметрами
        clf = LogisticRegression(**lm_params)
        
        model = Pipeline([
            ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
                transformer_list=[
                # Pipeline for pulling features from the post's subject line
                    ('scaler', Pipeline([
                            ('selector', ColumnSelector(col_idx=[2,3,4])),
                            ('scaler', scaler),
                        ])
                    ),
                    # Pipeline for standard bag-of-words model
                    ('text', Pipeline([
                            ('selector', ColumnSelector(col_idx=[5,])),
                            ('tfidf', TfidfVectorizer(
                                ngram_range=(1,1),
                                max_df=0.95, 
                                min_df=5)
                            ),
                        ])
                    ),
                ],# end  transformer_list
                # weight components in FeatureUnion
                transformer_weights={
                    'scaler': 0.8,
                    'text': 0.5,
                }) # end FeatureUnion
            ),# end union
            ('lr', clf),
        ]) # end pipeline
        
        # Схема кросс-валидации
        n_splits = 5
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, 
                             random_state=RND_SEED)
        
        scores = cross_val_score(model, X_, y,
                                 scoring='roc_auc', 
                                 cv=cv, 
                                 n_jobs=1)

        # Возвращаем среднее значение метрики и отклонение (на всякий случай)
        return scores.mean(), scores.std()

    def f(params):
        acc, std = hyperopt_cv(X, y, params)
        return {'loss': -acc, 'qscore': -acc, 'qscore_std': std, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(f, 
                params, 
                algo=tpe.suggest, 
                max_evals=evals, 
                trials=trials, 
                verbose=2)
    
    return trials

In [83]:
# Задаем пространство поиска
space4_lm = {
    'lm_penalty': hp.choice('penalty', ['l1', 'l2']),
    'lm_C': hp.loguniform('C', -5, 3),
    'lm_class_weight': hp.choice('class_weight', [None, 'balanced']),
    'lm_random_state': RND_SEED,
    'scaler_type': hp.choice('scaler_type', ['standart', 'robust']),
    'scaler_centering': hp.choice('scaler_centering', [False, True])
}

In [84]:
trials = run_trials_template(X, y, space4_lm, evals=40)

/home/ssmarkin/zwork/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [ ]:
def trials_df(trials):
    '''
    Функция форматирует результаты hyperopt в dataframe
    '''
    tr_dict = []
    for t in trials:
        trial = dict()
        for k, v in t['misc']['vals'].items():
            trial[k] = v[0]

        trial['qscore'] = -t['result']['qscore']
        trial['qscore_std'] = -t['result']['qscore_std']
        tr_dict.append(trial)

    df_res = pd.DataFrame.from_dict(tr_dict)
    df_res = df_res.sort_values('qscore', ascending=False)
    
    return df_res

In [405]:
df_trials = trials_df(trials)

In [407]:
# что-то посчитало и не упало. Я хз, ну хоть не упало и на том спасибо.

In [406]:
df_trials.head()

,C,class_weight,penalty,qscore,qscore_std,scaler_centering,scaler_type
28,0.113750,1,0,0.629355,-0.010612,0,1
32,0.125941,1,0,0.629353,-0.010570,0,1
29,0.120338,1,0,0.629349,-0.010564,0,1
27,0.094174,1,0,0.629256,-0.010703,0,1
0,0.215682,0,0,0.629162,-0.010488,0,1


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

a = [u'Мама мыла раму',
 u'Шла саша по шоссе',
 u'Корабли лавировали-лавировали']

TfidfVectorizer().fit_transform(a)

<3x9 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>